In [1]:
import os
import pandas as pd
import xml.etree.ElementTree as ET


def parse_xml_file(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    problems = root.findall("problem")

    problem_data = {}
    for problem in problems:
        problem_id = problem.find("problem_class").get("id")
        problem_data[problem_id] = problem_data.get(problem_id, 0) + 1

    return problem_data


def process_assignment(assignment_path):
    assignment_name = os.path.basename(assignment_path)
    student_folders = [f for f in os.listdir(assignment_path) if f.startswith("Student")]
    problem_names = set()
    student_data = {}
    for student_folder in student_folders:
        student_id = int(student_folder.split(" ")[1])
        student_files = os.listdir(os.path.join(assignment_path, student_folder))
        problem_counts = {}
        for file in student_files:
            if file.endswith(".xml"):
                xml_path = os.path.join(assignment_path, student_folder, file)
                problem_data = parse_xml_file(xml_path)
                for problem_name, count in problem_data.items():
                    problem_counts[problem_name] = problem_counts.get(problem_name, 0) + count
                    problem_names.add(problem_name)
        student_data[student_id] = problem_counts
    return assignment_name, student_data, problem_names


def create_table(assignment_paths):
    problem_names = set()
    table_data = {}
    for assignment_path in assignment_paths:
        assignment_name, student_data, assignment_problem_names = process_assignment(assignment_path)
        problem_names.update(assignment_problem_names)
        for student_id, problem_counts in student_data.items():
            if student_id not in table_data:
                table_data[student_id] = {}
            for problem_name, count in problem_counts.items():
                table_data[student_id][problem_name] = table_data[student_id].get(problem_name, 0) + count

    table_df = pd.DataFrame.from_dict(table_data, orient="index")
    table_df.index.name = "student"
    table_df = table_df.fillna(0)
    table_df = table_df.astype(int)
    table_df = table_df.reindex(sorted(table_df.columns), axis=1)

    return table_df


if __name__ == "__main__":
    assignment_paths = [f for f in os.listdir(".") if f.startswith("Homework")]
    table = create_table(assignment_paths)
    print(table)

            AccessStaticViaInstance  AnonymousClassComplexity  \
student                                                         
2        2                        0                         0   
4        1                        0                         2   
6        1                        0                         0   
8        1                        0                         0   
9        1                        0                         0   
10       1                        0                         1   
11       2                        0                         1   
12       1                        0                         1   
13       3                        0                         0   
14       4                        0                         2   
18       2                        0                         3   
19       1                        0                         1   
20       1                        0                         0   
22       1               

In [2]:
table

AccessStaticViaInstance  AnonymousClassComplexity  \
student                                                         
2        2                        0                         0   
4        1                        0                         2   
6        1                        0                         0   
8        1                        0                         0   
9        1                        0                         0   
10       1                        0                         1   
11       2                        0                         1   
12       1                        0                         1   
13       3                        0                         0   
14       4                        0                         2   
18       2                        0                         3   
19       1                        0                         1   
20       1                        0                         0   
22       1                        0                         0   
23       1                        0                         3   
31       2                        0                         0   
32       1                        0                         0   
34       1                        3                         0   
37       1                        0                         0   
38       1                        0                         0   
40       1                        0                         0   
41       1                        0                         1   
44       1                        0                         0   
45       1                        0                         0   
5        0                        0                         0   

         AnonymousClassMethodCount  BooleanMethodIsAlwaysInverted  \
student                                                             
2                                0                              0   
4                                0                              0   
6                                0                              0   
8                                0                              0   
9                                0                              0   
10                               0                              0   
11                               1                              0   
12                               0                              0   
13                               0                              0   
14                               0                              0   
18                               3                              0   
19                               0                              0   
20                               0                              1   
22                               0                              0   
23                               0                              1   
31                               0                              0   
32                               0                              0   
34                               0                              0   
37                               0                              0   
38                               0                              0   
40                               0                              0   
41                               1                              0   
44                               0                              0   
45                               0                              0   
5                                0                              0   

         CStyleArrayDeclaration  CanBeFinal  CatchMayIgnoreException  \
student                                                                
2                             0          16                        0   
4                             0          14                        0   
6                             0          12                        0   
8                             0 

In [6]:
# Compute the sum of each column
sums = table.sum()

max_columns = sums.nlargest(10).index.tolist()

# Print the results
print('Sum of each column:')
print(sums)
print('Top 10 columns with the highest sum:', max_columns)

Sum of each column:
                                  33
AccessStaticViaInstance            3
AnonymousClassComplexity          15
AnonymousClassMethodCount          5
BooleanMethodIsAlwaysInverted      2
                                ... 
UnusedAssignment                  17
UnusedReturnValue                  6
UtilityClass                      20
WrongPackageStatement            187
unused                           542
Length: 79, dtype: int64
Top 10 columns with the highest sum: ['MissingJavadoc', 'unused', 'SingleClassImport', 'CanBeFinal', 'WrongPackageStatement', 'MultipleReturnPointsPerMethod', 'MethodWithMultipleLoops', 'FieldMayBeFinal', 'ChainedMethodCall', 'FeatureEnvy']


In [5]:
codeSmells=table[max_columns]
codeSmells

MissingJavadoc  unused  SingleClassImport  CanBeFinal  \
student                                                          
2                   102      71                 14          16   
4                    34      37                 40          14   
6                    51      15                 12          12   
8                    32      15                  2           3   
9                    38      37                  4           8   
10                   23      24                 36          10   
11                   66      17                  4           4   
12                   18       8                  7          20   
13                   33       2                  7          12   
14                   47      28                 18          31   
18                   86      39                 38           7   
19                   46      32                 28          11   
20                   33      18                  7          19   
22                   27       8                  5           6   
23                   34       6                 12           2   
31                   60      30                 27          22   
32                   31      14                  9          18   
34                   24      32                  9          11   
37                   20       9                 13           6   
38                   48      24                 28           5   
40                    8       3                  5           8   
41                   16      14                  5           3   
44                   31      14                  9          18   
45                   19      14                 19           3   
5                    30      31                  5           0   

         WrongPackageStatement  MultipleReturnPointsPerMethod  \
student                                                         
2                           11                              7   
4                           10                              5   
6                            8                              8   
8                            4                              5   
9                            7                              5   
10                           8                              5   
11                           8                              9   
12                           9                              8   
13                          12                              9   
14                          11                             10   
18                          11                             18   
19                           9                             11   
20                           8                              3   
22                           4                              3   
23                           6                              5   
31                           7                             26   
32                           6                              8   
34                           6                              2   
37                           5                              3   
38                          12                             12   
40                           2                              1   
41                           5                              5   
44                           6                              8   
45                           9                              2   
5                            3                              2   

         MethodWithMultipleLoops  FieldMayBeFinal  ChainedMethodCall  \
student                                                                
2                              9               10                  0   
4                              7               12                  4   
6                              9               10                  3   
8                              3                0                  3   
9                              5      

In [15]:
codeSmells.to_csv('hw4.csv', index=False)